##### Primary Notion <=> in-degree == 0 & out-degree > 0
##### Learning Outcome <=> in-degree > 0 & out-degree == 0

In [18]:
import networkx as nx
import igraph as igraph
import pandas as pd
import numpy as np
from networkx.readwrite import json_graph
import json
import codecs
import csv
import ast

In [19]:
# intialise data of leafs
# leafs_dic={'concept':[],'frosina':[],'chiara':[],'samuele':[],'andrea':[],'marco':[],'ilenia':[]}
temporal_leafs_df = pd.DataFrame(columns=['concept','frosina','chiara','samuele','andrea', 'marco', 'ilenia'])
print(temporal_leafs_df)

# intialise data of roots
# leafs_dic={'concept':[],'frosina':[],'chiara':[],'samuele':[],'andrea':[],'marco':[],'ilenia':[]}
temporal_roots_df = pd.DataFrame(columns=['concept','frosina','chiara','samuele','andrea', 'marco', 'ilenia'])

temporal_leafs_roots_df = pd.DataFrame(columns=['concept','frosina','chiara','samuele','andrea', 'marco', 'ilenia'])

Empty DataFrame
Columns: [concept, frosina, chiara, samuele, andrea, marco, ilenia]
Index: []


In [20]:
combined_terminology_set = set()
with open('../input_data/annotazioni2/frosina.json', 'r', encoding='utf-8') as json_data:
    parsed_json = (json.loads(json_data.read()))
    for internal_dic in parsed_json["savedInsertedRelations"]:
        combined_terminology_set.add(internal_dic["prerequisite"])
        combined_terminology_set.add(internal_dic["advanced"]) 
with open('../input_data/annotazioni2/chiara.json', 'r', encoding='utf-8') as json_data:
    parsed_json = (json.loads(json_data.read()))
    for internal_dic in parsed_json["savedInsertedRelations"]:
        combined_terminology_set.add(internal_dic["prerequisite"])
        combined_terminology_set.add(internal_dic["advanced"])
with open('../input_data/annotazioni2/samuele.json', 'r', encoding='utf-8') as json_data:
    parsed_json = (json.loads(json_data.read()))
    for internal_dic in parsed_json["savedInsertedRelations"]:
        combined_terminology_set.add(internal_dic["prerequisite"])
        combined_terminology_set.add(internal_dic["advanced"])
with open('../input_data/annotazioni2/AndreaGarrone.json', 'r', encoding='utf-8') as json_data:
    parsed_json = (json.loads(json_data.read()))
    for internal_dic in parsed_json["savedInsertedRelations"]:
        combined_terminology_set.add(internal_dic["prerequisite"])
        combined_terminology_set.add(internal_dic["advanced"])
with open('../input_data/annotazioni2/MarcoMochi.json', 'r', encoding='utf-8') as json_data:
    parsed_json = (json.loads(json_data.read()))
    for internal_dic in parsed_json["savedInsertedRelations"]:
        combined_terminology_set.add(internal_dic["prerequisite"])
        combined_terminology_set.add(internal_dic["advanced"])
        
print('combined_terminology_set len:',len(combined_terminology_set))
temporal_leafs_df['concept']=list(combined_terminology_set)
leafs_df = temporal_leafs_df.set_index('concept', verify_integrity=True)
temporal_roots_df['concept']=list(combined_terminology_set)
roots_df = temporal_roots_df.set_index('concept', verify_integrity=True)
temporal_leafs_roots_df['concept']=list(combined_terminology_set)
leafs_roots_df = temporal_leafs_roots_df.set_index('concept', verify_integrity=True)

combined_terminology_set len: 354


### Get Annotations from json file & create graph
#### Run it for every annotator json

In [32]:
# frosina.json
# chiara.json
# samuele.json
# AndreaGarrone.json
# MarcoMochi.json
# ILENIA_2019-m11-d25-h12-m46-s35.json
with open('../input_data/annotazioni2/MarcoMochi.json', 'r', encoding='utf-8') as json_data:
    parsed_json = (json.loads(json_data.read()))
    # The terminology set is the set terminology of all the annotators
    terminology_set = set()
    for internal_dic in parsed_json["savedInsertedRelations"]:
        terminology_set.add(internal_dic["prerequisite"])
        terminology_set.add(internal_dic["advanced"])
    # print(terminology_set) 
    print('# of prereq rel:', len(parsed_json["savedInsertedRelations"]))
    print('terminology len:',len(terminology_set))
    
    networkx_graph = nx.DiGraph()
    igraph_graph = igraph.Graph(directed=True)
    
    networkx_graph.add_nodes_from(terminology_set)
    igraph_graph.add_vertices(terminology_set)
    terminology_list = list(terminology_set)
    
    for internal_dic in parsed_json["savedInsertedRelations"]:
        prereq = internal_dic["prerequisite"]
        subsid = internal_dic["advanced"]
        networkx_graph.add_edge(prereq, subsid)
        igraph_graph.add_edge(terminology_list.index(prereq), terminology_list.index(subsid))


print("# rel",networkx_graph.size())
print("# nodes",networkx_graph.number_of_nodes())
len(networkx_graph)

# len(igraph_graph1.es)
igraph.Graph.summary(igraph_graph)

# of prereq rel: 282
terminology len: 141
# rel 275
# nodes 141


'IGRAPH DN-- 141 282 -- \n+ attr: name (v)'

 ## In-degree, Out-degree calcs

In [33]:
    # size
    num_nodes = len(nx.nodes(networkx_graph))
    num_edges = len(nx.edges(networkx_graph))
    
    # degrees
    max_out_degree =igraph.GraphBase.maxdegree(igraph_graph, mode="OUT")

    out_degree_dict = {}
    in_degree_dict = {}

    for (node, val) in networkx_graph.out_degree():
        if val > 5:
            out_degree_dict[node]=val

    for (node, val) in networkx_graph.in_degree():
        if val > 1:
            in_degree_dict[node]=val

    # print("out degree", repr(out_degree_dict))
    # print("in degree", repr(in_degree_dict))

    with open('out_degree.csv', 'w') as f:
        for key in out_degree_dict.keys():
            f.write("%s,%s\n" % (key, out_degree_dict[key]))

    with open('in_degree.csv', 'w') as f:
        for key in in_degree_dict.keys():
            f.write("%s,%s\n" % (key, in_degree_dict[key]))
  
    leafs = [{x:networkx_graph.in_degree(x)} for x in networkx_graph.nodes() if networkx_graph.out_degree(x)==0 and networkx_graph.in_degree(x)>=1]
    for item in leafs:
        leafs_df.loc[next(iter(item)), 'marco'] = item[next(iter(item))]
        leafs_roots_df.loc[next(iter(item)), 'marco'] = item[next(iter(item))]
    
    
    roots = [{x:networkx_graph.out_degree(x)}  for x in networkx_graph.nodes() if networkx_graph.in_degree(x)==0 and networkx_graph.out_degree(x)>=1]
    for item in roots:
        # set negative numbers for the # of out-degree arcs
        roots_df.loc[next(iter(item)), 'marco'] = item[next(iter(item))]
        leafs_roots_df.loc[next(iter(item)), 'marco'] = -item[next(iter(item))]

In [34]:
### Create File CSV
 
#print(leafs_df)
leafs_df.to_csv (r'export_LOs.csv', index = True, header=True)

# print(roots_df)
roots_df.to_csv (r'export_PNs.csv', index = True, header=True)

print(leafs_roots_df)
leafs_roots_df.to_csv (r'export_PNs_LOs.csv', index = True, header=True)

                                                   frosina chiara samuele  \
concept                                                                     
APPLET                                                   3    NaN       2   
TRANSMISSION_CONTROL_PROTOCOL                          NaN      1     NaN   
TRANSMISSION_CONTROL_PROTOCOL/INTERNET_PROTOCOL...     NaN    NaN     NaN   
FILE_SHARING                                           NaN    NaN       1   
CERTIFICATE                                              2    NaN       1   
PROPRIETARY_SYSTEM                                     NaN    NaN      -4   
PRIVACY                                                 -2    NaN     NaN   
REGISTRAR                                              NaN    NaN       1   
3G                                                       2      2       2   
SEVEN-LEVEL_HIERARCHY                                  NaN    NaN       1   
MASS_STORAGE                                           NaN     -1     NaN   

In [35]:
print("summary statistic leafs",leafs_df.describe())
print("mean", leafs_df.mean())
print("corr", leafs_df.corr())
print("# of non null", leafs_df.count())
print("standard deviation", leafs_df.std())

summary statistic leafs         frosina  chiara  samuele  andrea  marco  ilenia
count        76      46      101     100     62     0.0
unique        6       7        7       7      6     0.0
top           1       2        1       1      2     NaN
freq         26      14       48      38     21     NaN
mean frosina    2.328947
chiara     2.695652
samuele    1.920792
andrea     2.040000
marco      2.306452
ilenia          NaN
dtype: float64
corr Empty DataFrame
Columns: []
Index: []
# of non null frosina     76
chiara      46
samuele    101
andrea     100
marco       62
ilenia       0
dtype: int64
standard deviation frosina    1.340529
chiara     1.836295
samuele    1.205680
andrea     1.222020
marco      1.325521
ilenia          NaN
dtype: float64


In [36]:
print("summary statistic roots",roots_df.describe())
print("mean", roots_df.mean())
print("corr", roots_df.corr())
print("# of non null", roots_df.count())
print("standard deviation", roots_df.std())

summary statistic roots         frosina  chiara  samuele  andrea  marco  ilenia
count        58      24       69      28     26     0.0
unique       10       8       11      13      6     0.0
top           1       1        1       1      1     NaN
freq         21      14       36       8     12     NaN
mean frosina    3.327586
chiara     4.166667
samuele    2.753623
andrea     6.285714
marco      3.461538
ilenia          NaN
dtype: float64
corr Empty DataFrame
Columns: []
Index: []
# of non null frosina    58
chiara     24
samuele    69
andrea     28
marco      26
ilenia      0
dtype: int64
standard deviation frosina    3.278788
chiara     6.126682
samuele    3.331692
andrea     7.639358
marco      4.456283
ilenia          NaN
dtype: float64


In [37]:
leafs_roots_df.astype(float)
df_0=leafs_roots_df.fillna(0)
#print(df_0)
print(df_0.corr(method ='pearson'))
print(df_0.corr(method ='kendall'))
print(df_0.corr(method ='spearman'))

          frosina    chiara   samuele    andrea     marco  ilenia
frosina  1.000000  0.502648  0.562756  0.316589  0.560976     NaN
chiara   0.502648  1.000000  0.418927  0.409685  0.697457     NaN
samuele  0.562756  0.418927  1.000000  0.419566  0.371946     NaN
andrea   0.316589  0.409685  0.419566  1.000000  0.368816     NaN
marco    0.560976  0.697457  0.371946  0.368816  1.000000     NaN
ilenia        NaN       NaN       NaN       NaN       NaN     NaN
          frosina    chiara   samuele    andrea     marco  ilenia
frosina  1.000000  0.251083  0.333624  0.096109  0.290273     NaN
chiara   0.251083  1.000000  0.278790  0.295086  0.406904     NaN
samuele  0.333624  0.278790  1.000000  0.180666  0.241585     NaN
andrea   0.096109  0.295086  0.180666  1.000000  0.262171     NaN
marco    0.290273  0.406904  0.241585  0.262171  1.000000     NaN
ilenia        NaN       NaN       NaN       NaN       NaN     1.0
          frosina    chiara   samuele    andrea     marco  ilenia
frosina  1

In [ ]:
roots_df.astype(float)
df_roots_0=roots_df.fillna(0)
#print(df_0)
print(df_roots_0.corr(method ='pearson'))
print(df_roots_0.corr(method ='kendall'))
print(df_roots_0.corr(method ='spearman'))

In [ ]:
leafs_df.astype(float)
df_leafs_0=leafs_df.fillna(0)
#print(df_0)
print(df_leafs_0.corr(method ='pearson'))
print(df_leafs_0.corr(method ='kendall'))
print(df_leafs_0.corr(method ='spearman'))

In [37]:
### Initialize

          frosina    chiara   samuele    andrea     marco  ilenia
frosina  1.000000  0.563050  0.536119  0.362470  0.665114     NaN
chiara   0.563050  1.000000  0.418964  0.380818  0.835807     NaN
samuele  0.536119  0.418964  1.000000  0.456579  0.364967     NaN
andrea   0.362470  0.380818  0.456579  1.000000  0.358915     NaN
marco    0.665114  0.835807  0.364967  0.358915  1.000000     NaN
ilenia        NaN       NaN       NaN       NaN       NaN     NaN
          frosina    chiara   samuele    andrea     marco  ilenia
frosina  1.000000  0.226214  0.086681  0.116420  0.316033     NaN
chiara   0.226214  1.000000  0.123328  0.255797  0.573452     NaN
samuele  0.086681  0.123328  1.000000  0.186674  0.203627     NaN
andrea   0.116420  0.255797  0.186674  1.000000  0.243899     NaN
marco    0.316033  0.573452  0.203627  0.243899  1.000000     NaN
ilenia        NaN       NaN       NaN       NaN       NaN     1.0
          frosina    chiara   samuele    andrea     marco  ilenia
frosina  1

In [38]:
leafs_df.astype(float)
df_leafs_0=leafs_df.fillna(0)
#print(df_0)
print(df_leafs_0.corr(method ='pearson'))
print(df_leafs_0.corr(method ='kendall'))
print(df_leafs_0.corr(method ='spearman'))

          frosina    chiara   samuele    andrea     marco  ilenia
frosina  1.000000  0.277493  0.504072  0.024313  0.243412     NaN
chiara   0.277493  1.000000  0.332779  0.471413  0.356157     NaN
samuele  0.504072  0.332779  1.000000  0.124101  0.249624     NaN
andrea   0.024313  0.471413  0.124101  1.000000  0.344117     NaN
marco    0.243412  0.356157  0.249624  0.344117  1.000000     NaN
ilenia        NaN       NaN       NaN       NaN       NaN     NaN
          frosina    chiara   samuele    andrea     marco  ilenia
frosina  1.000000  0.230630  0.382257  0.024037  0.202765     NaN
chiara   0.230630  1.000000  0.308206  0.342331  0.345145     NaN
samuele  0.382257  0.308206  1.000000  0.089014  0.175743     NaN
andrea   0.024037  0.342331  0.089014  1.000000  0.323269     NaN
marco    0.202765  0.345145  0.175743  0.323269  1.000000     NaN
ilenia        NaN       NaN       NaN       NaN       NaN     1.0
          frosina    chiara   samuele    andrea     marco  ilenia
frosina  1

### Initialize

In [ ]:
# G_nx, G_ig, annotator = create_graphs('datasets/prova_relazione_tool.csv', dataset)
# metrics = compute_metrics(G_nx, G_ig)
# detected_cycles = detect_loops(G_nx, G_ig)
# trans_dict = detect_transitive_edges(G_nx, cutoff=metrics['diameter'])
# membership = detect_clusters(G_ig)
# graph_dict = create_graph_dict(dataset, gold_with_annotators, annotator, metrics, detected_cycles, trans_dict, membership)
# output_json = export_to_json("output_files//prova.json", graph_dict)




